# Scraping Google Trends data using Organisation for Economic Co-operation and Development (OECD) keywords

 

In [1]:
# import libraries/modules
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=360, timeout=(40,25))
import json

## Load the keyword dictionary 

In [3]:
oecd_key = pd.read_csv('data/keywords/OECD_keyword.csv')
oecd_key.head()

,category_name
0,Economic crisis
1,Crisis
2,Recession
3,Financial crisis
4,Krach


## Get all the keywords and create a list 

In [4]:
oecd_key = oecd_key["category_name"].values.tolist()
oecd_key

['Economic crisis',
 'Crisis',
 'Recession',
 'Financial crisis',
 'Krach',
 'Unemployment',
 'Unemployment benefits',
 'Welfare & Unemployment',
 'Food & Drink',
 'GPS & Navigation ',
 'Performing Arts             ',
 'Luggage topic',
 'Vehicle',
 'Brands',
 'Birthday',
 'Travel',
 'Energy & Utilities',
 'Vehicle Shopping',
 'Tobacco Products',
 'Health',
 'Pharmacy',
 'Carpooling & Ridesharing',
 'Sports',
 'Animal Products & Services',
 'Fitness',
 'Weddings',
 'Car',
 'Rental & Taxi Services',
 'Autos & Vehicles',
 'Tourist Destinations',
 'Home & Garden',
 'Events & Listings',
 'Grocery & Food Retailers',
 'Vehicle Licensing & Registration',
 'Timeshares & Vacation Properties',
 'Home',
 'Appliances',
 'Mass Merchants & Department Stores',
 'Car Electronics',
 'Fashion & Style',
 'Trucks & SUVs',
 'Home Furnishings',
 'Footwear',
 'Cruises & Charters',
 'Hotels & Accommodations',
 'Luggage & Travel',
 'Accessories',
 'Fast Food',
 'Book Retailers',
 'Veterinarians',
 'Spas & Beaut

## Remove duplicates from a list

In [5]:
keywords = list(set(oecd_key))
len(keywords)

134

## Define the chunk size and split the list into chunks

In [6]:
# Define the chunk size
chunk_size = 15

# Split the list into chunks
chunks = [keywords[i:i + chunk_size] for i in range(0, len(keywords), chunk_size)]

len(chunks)

9

## Name each chunk


In [7]:
# Name each chunk as chunk_i starting from 1
for i, chunk in enumerate(chunks, 1):
    globals()[f'chunks{i}'] = chunk

## Define a function fetches the Google Trend data

In [25]:
def fetch_trends_in_batches(chunks,
                            batch_size=1,
                            cat=0, 
                            timeframe='2004-01-01 2024-03-31', 
                            geo='MU', 
                            gprop=''):
    
    pytrends = TrendReq(hl='en-US', tz=360, timeout=(40, 25))
    
    all_data = []
    num_batches = -(-len(chunks) // batch_size)  # Calculate number of batches
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        batch_keywords = chunks[start_idx:end_idx]
        
        pytrends.build_payload(batch_keywords, cat=cat, 
                               timeframe=timeframe,
                               geo=geo, 
                               gprop=gprop)
        data = pytrends.interest_over_time()
        
        all_data.append(data)

    combined_data = pd.concat(all_data, axis=1)
    return combined_data

## Store combined dataframes in a dictionary

In [26]:
combined_data_dict = {}

batch_size = 1
for i in range(1, len(chunks) + 1):
    chunks = globals()[f'chunks{i}']
    combined_data_dict[f'combined_data{i}'] = fetch_trends_in_batches(chunks, 
                                                                      batch_size=batch_size, 
                                                                      cat=0, 
                                                                      timeframe='2004-01-01 2024-03-31', 
                                                                      geo='MU', 
                                                                      gprop='')
len(combined_data_dict)

KeyError: 'chunks10'

## Combine and stored the dataframes 

In [27]:
# Determine the number of dataframes stored in combined_data_dict
num_dataframes = len(combined_data_dict)

# Initialize an empty list to store dataframes
dataframes = []

# Loop through the keys to access each dataframe
for i in range(1, num_dataframes + 1):
    key = f'combined_data{i}'
    if key in combined_data_dict:
        dataframes.append(combined_data_dict[key])

# Concatenate the dataframes along the date index
combined_df = pd.concat(dataframes, axis=1)

combined_df = combined_df.loc[:, ~combined_df.columns.str.endswith('isPartial')]

# Reset index to make date index
combined_df.reset_index(inplace=True)

# Drop duplicate date columns
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

# Set date column as index
combined_df.set_index('date', inplace=True)

combined_df

,Waiter,Recruitment,Acting & Theater,Foreclosure,Brands,Birthday,Tourist Destinations,Home,Insurance,Economy News,...,Engineering,Events & Listings,Development Tools,Private employment agency,Recession,Politics,Aviation,Financial crisis,Computer Security,Manufacturing
date,,,,,,,,,,,,,,,,,,,,,
2004-01-01,0,0,0,0,0,0,0,100,0,0,...,0,0,0,0,0,0,0,0,0,0
2004-02-01,0,0,0,0,0,0,0,0,0,0,...,100,0,0,0,0,0,0,0,0,0
2004-03-01,0,0,0,0,0,0,0,93,0,0,...,0,0,0,0,0,0,0,0,0,0
2004-04-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2004-05-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-01,0,12,0,0,6,56,0,31,65,0,...,5,0,0,0,0,0,30,0,0,9
2023-12-01,31,11,0,0,5,54,0,34,56,0,...,4,0,0,0,0,15,28,0,0,10
2024-01-01,33,14,0,0,6,49,0,30,57,0,...,5,0,0,0,0,0,25,0,0,10


## Export the dataframe


In [28]:
# Export the data to a CSV file
combined_df.to_csv('data/gt_MU_OECD.csv')